# Federated learning on images

In [ ]:
import os
import pydicom
import radiomics

import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import log_loss
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay

from src.helper import *

import warnings
warnings.filterwarnings('ignore')

## Data pre-processing

We are going to start, as in any other data analytics project, with pre-processing the data to prepare it for modelling. The input data contains CT scans of head & neck patients and clinical data related to them. Radiomic features will be extracted from the CT scans and used as predictors and distance metastasis events will be used as the outcome that we will try to predict.

In [ ]:
# Data path
data_path = os.path.join(os.getcwd(), 'data')

### Clinical data

In [ ]:
# EXERCISE
# Read the `opc_nodeX_data.csv` file, visualise the first rows, and information about the data
# Node 1: `opc_node1_data.csv`
# Node 2: `opc_node2_data.csv`


- What type of data is included?
- How many rows and columns?
- Can you identify the distance metastasis event column?

We are now going to select the patients we are going to use for further analysis. We want to use non-metastatic p16-positive patients and are going to keep the distance metastasis event as outcome.

In [ ]:
# EXERCISE
# Filter the data by selecting only the patients we plan to use for further analysis


- How did you filter the data for the patient selection criteria?
- How many patients are left?

In [ ]:
# EXERCISE
# How many patients are available for the federated analysis?
# We are now going to simulate a federated calculation with one iteration
# patients_node1 = ?
# patients_node2 = ?


In [ ]:
# EXERCISE
# Replace the distance metastasis events with binary values


### Radiotherapy planning CT scans

We are now going to extract radiomic features from the CT scans to use as predictors.

#### Extracting radiomic features for a single patient

In [ ]:
# CT scans path
data_path_scans = os.path.join(data_path, 'CT_scans')

In [ ]:
# Get CT image
id = os.listdir(data_path_scans)[0]
slices_path = os.path.join(data_path_scans, id, 'DICOM')
slices = load_scan(slices_path)
image = get_ct_image(slices)

In [ ]:
# Get binary mask
rt_path = os.path.join(data_path_scans, id, 'RTSTRUCT')
rt_file = os.listdir(rt_path)[0]
rt_path = os.path.join(rt_path, rt_file)
mask = get_gtv_mask(slices, rt_path)

In [ ]:
# Get radiomic features
params = os.path.join(os.getcwd(), 'params', 'pyradiomics_params_all.yaml')
extractor = radiomics.featureextractor.RadiomicsFeatureExtractor(params)
features = pd.Series(extractor.execute(image, mask))
print(features.tail())

Explore the information in the `features` series:

- Which types of features were extracted?
- Besides the features, which other information is available?
- Have a look at the `pyradiomics_params_all.yaml` file, which is used to define what will be extracted

#### Extracting the radiomic features for all patients

As you could see above, `pyradiomics` can extract a multitude of features from the CT scans based on the gross tumour volume (GTV) mask. In order to keep our analysis simple, we are going to select only 5 features: 

- Surface Area (shape)
- Energy (first order)
- Cluster Prominence (GLCM)
- Large Area Emphasis (GLSZM)
- Gray Level Non Uniformity (GLRLM)

Go to the `pyradiomics_params.yaml` file in the `params` directory and edit it to extract only the 5 above features.

In [ ]:
%%time
patient_ids = list(df_clinical['Trial PatientID'].values)
df_features = pd.DataFrame()
for id in patient_ids:
    try:
        print(f'Extracting radiomics for patient {id}')
    
        # Get CT image
        slices_path = os.path.join(data_path_scans, id, 'DICOM')
        slices = load_scan(slices_path)
        image = get_ct_image(slices)
    
        # Get binary mask
        rt_path = os.path.join(data_path_scans, id, 'RTSTRUCT')
        rt_file = os.listdir(rt_path)[0]
        rt_path = os.path.join(rt_path, rt_file)
        mask = get_gtv_mask(slices, rt_path)
    
        # Get radiomic features
        params = os.path.join(os.getcwd(), 'params', 'pyradiomics_params.yaml')
        extractor = radiomics.featureextractor.RadiomicsFeatureExtractor(params)
        features = pd.Series(extractor.execute(image, mask))
        
        # Organise features in a dataframe
        df = pd.DataFrame(features[47:]).T
        df['Trial PatientID'] = id
        df_features = pd.concat([df_features, df], ignore_index=True)
    except:
        print(f'No DICOM available for patient {id}')

df_features.head()

## Prepare input data for modelling

In [ ]:
# EXERCISE
# Merge features and outcomes dataframes


In [ ]:
# EXERCISE
# Check counts of distant metastasis


- What have you noticed?

In [ ]:
# Normalise features
columns = [
    'original_shape_SurfaceArea', 'original_firstorder_Energy', 'original_glcm_ClusterProminence', 
    'original_glszm_LargeAreaEmphasis', 'original_glrlm_GrayLevelNonUniformity'
]
norms = [32807, 18721084189, 1686168, 270858, 2577]
for i in range(len(norms)):
    df[columns[i]] = df[columns[i]]/norms[i]

Notice that you were given normalization constants. In a federated setting the nodes would need to exchange their maximums and minimum and then obtain the overall maximum and minimum. Another option would be to use a common constant, which would not give the guarantee that the variables would be between 0 and 1.

In [ ]:
# Get features (X) and outcomes (y)
X = df[columns].values
y = df['Distant Failure'].values

## Train federated logistic regression

We are now going to simulate a few federated training rounds, so that you can get a better intuition on how it works and how to design the algorithms.

### Round 1

In [ ]:
# Data node task
# Create local logistic regression model
model = LogisticRegression(max_iter=1, warm_start=True)

In [ ]:
# Data node task, but initial values are received from central server
# Set initial guess
model.coef_ = np.array([[0.19947649,  0.33157079, -0.00131289,  0.23688854,  0.26542626]])
model.intercept_ = np.array([-0.96108667])

In [ ]:
# Data node task
# Fit local model with the training data
model.fit(X, y)

In [ ]:
# Check local coefficients
print(f'Slopes: {model.coef_}')
print(f'Intercept: {model.intercept_}')

In [ ]:
# Central server procedure
# Function to aggregate local coefficients into a global model
def aggregate_coefficients(wj, nj):
    w = np.zeros(len(wj[0]))
    N = np.sum(nj)
    for p in range(len(wj[0])):
        for j in range(len(nj)):
            w[p] += nj[j]*wj[j, p]
    w = w/N
    return w

In [ ]:
# EXERCISE
# Central server task
# Aggregate coefficients, we are now going to exchange the local coefficients
nj = np.array([50, 55])
wj = np.array([
    [<NODE_1_COEFFICIENTS>],
    [<NODE_1_COEFFICIENTS>]
])
w = aggregate_coefficients(wj, nj)
print(f'Round 1')
for i in range(len(w)):
    print(f'w{i} = {w[i]}')

In [ ]:
# Central server task, result is sent to data nodes
# Update global model
model.coef_ = np.array([w[1:]])
model.intercept_ = np.array([w[0]])

In [ ]:
# Data node task
# Compute local loss
loss = log_loss(y, model.predict_proba(X))
print(f'Local loss: {loss}')

Since we are simulating a few iterations, we are not going to combine the local losses, but we will keep track of how it is evolving.

### Round 2

In [ ]:
# EXERCISE
# Data node task
# Fit local model with the training data and new guess for initial coefficients


In [ ]:
# EXERCISE
# Check local coefficients


In [ ]:
# EXERCISE
# Central server task
# Aggregate coefficients, we are now going to exchange the local coefficients


In [ ]:
# EXERCISE
# Central server task, result is sent to data nodes
# Update global model


In [ ]:
# EXERCISE
# Data node task
# Compute local loss


### Round 3

In [ ]:
# EXERCISE
# Data node task
# Fit local model with the training data and new guess for initial coefficients


In [ ]:
# EXERCISE
# Check local coefficients, we are now going to exchange the local coefficients


In [ ]:
# EXERCISE
# Central server task
# Aggregate coefficients, we are now going to exchange the local coefficients


In [ ]:
# EXERCISE
# Central server task, result is sent to the researcher as this is the breaking criteria (3 rounds)
# Update global model


In [ ]:
# EXERCISE
# Data node task
# Compute local loss


- How did the local loss evolve?

## Evaluating global model

Some data was kept in a simulated validation node. This is the method we are using to evaluate the final federated model.

In [ ]:
# Read data
file_path = os.path.join(data_path, 'opc_test_data.csv')
df_test = pd.read_csv(file_path)

In [ ]:
# EXERCISE
# Get features and outcomes (X_test, y_test)


In [ ]:
# Data node task, global model received from the central server
# Evaluate model by computing the accuracy
print(f'Accuracy: {model.score(X_test, y_test)}')

- What do you think of this result?

In [ ]:
# Confusion matrix
cm = confusion_matrix(y_test, model.predict(X_test), labels=model.classes_)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=model.classes_)
disp.plot()
plt.show()

## Centralised model

Finally, let's compare the federated solution to the centralised one.

In [ ]:
# Read data
file_path = os.path.join(data_path, 'opc_central_reduced_data.csv')
df_all = pd.read_csv(file_path)

In [ ]:
# EXERCISE
# Get features and outcomes (X, y)


In [ ]:
# EXERCISE
# Split data into train (80%) and test (20%) sets and use random_state=42
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=<ADD_SIZE>, random_state=42
)

In [ ]:
# EXERCISE
# Train centralised model using the default hyper-parameters  and visualise coefficients


- How do the coefficients compare?

In [ ]:
# EXERCISE
# Evaluate central model


In [ ]:
# EXERCISE
# Confusion matrix


- How do the performances from federated vs. centralised solutions compare?

## References

- [Pydicom: General examples](https://pydicom.github.io/pydicom/stable/auto_examples/index.html)
- [Pyradiomics documentation](https://pyradiomics.readthedocs.io/en/latest/index.html)
- [Ontology-guided Radiomics Analysis Workflow (O-RAW)](https://github.com/zhenweishi/O-RAW)
- [Kwan et al. (2018)](https://doi.org/10.1016/j.ijrobp.2018.01.057)